# Exploratory Data Analysis, Data Cleaning, NLP

In this notebook, I explore and clean the scraped data and use NLP techniques to prepare the comment text for modeling.

##  Importing modules and data

In [1]:
import pandas as pd
import numpy as np
import random

random.seed(32)

In [2]:
df = pd.read_csv('../data/nbacomments_raw.csv', index_col=False)

In [3]:
df.head()

,affiliation,text,thread_id,ups,when_comment_posted
0,Lakers1,Washington a fool for attacking Canada in the ...,t3_8ca8gj,132,1.523745e+09
1,Raptors2,Fun fact: CJ Miles has more 3s in this quarter...,t3_8ca8gj,82,1.523745e+09
2,Bucks2,Casey uses Bebe like a fucking Yu-Gi-Oh trap c...,t3_8ca8gj,78,1.523749e+09
3,Suns3,Ian Mahinmi is one of those players that you h...,t3_8ca8gj,73,1.523743e+09
4,Celtics1,Holy fuck the ACC is loud godamn,t3_8ca8gj,72,1.523743e+09


In [4]:
df['thread_id'] = df['thread_id'].map(lambda x: x.replace('t3_','')).copy()

Before we start, let's drop all deleted comments:

In [5]:
df = df[df['text'] != '[deleted]']

## EDA/Data Cleaning

To answer our question, we need to add to our dataframe which teams were playing in the games for each thread (labeled as Team1 and Team2):

In [6]:
thread_ids = pd.DataFrame(df['thread_id'].value_counts())

In [7]:
thread_ids.rename(columns={'thread_id' : 'comments'}, inplace=True)

In [8]:
threads_to_drop = list(thread_ids[thread_ids['comments'] < 500].index)

Based on the value counts of the thread IDs, I determined that there were 25 threads with fewer than 500 comments (where as the remaining threads all had more than 500 comments.) These threads with low comment count were deleted because they were created in error or were duplicates of an already existing thread. Let's drop those comments since we are only interested in comments in the official game threads:

In [9]:
df = df[~df['thread_id'].isin(threads_to_drop)]

Now we use those thread IDs and look up the teams playing in each game:

In [10]:
df.thread_id.unique()

array(['8ca8gj', '8cb74z', '8cc2gq', '8cg1l3', '8ch3y4', '8cifug',
       '8cje37', '8crv25', '8csumg', '8d0jib', '8d0vrr', '8d1xau',
       '8d9kql', '8da05s', '8daltu', '8dijtk', '8djcz2', '8djk1s',
       '8drdyv', '8drs6h', '8dsbrb', '8dx7i6', '8dy6uc', '8dz4uf',
       '8e00qg', '8e44h7', '8e55jb', '8e665k', '8e74eg', '8eft8n',
       '8egqf2', '8eod1l', '8eosom', '8epudl', '8exiel', '8exigg',
       '8eyk43', '8eysfu', '8ffe7x', '8ffrp7', '8fgnxe', '8fnfsr',
       '8foaac', '8fs7no', '8ft863', '8g3r8e', '8gcq8c', '8gdo7c',
       '8glq4c', '8gtqly', '8gut4j', '8h3dc3', '8h4a1y', '8ha1um',
       '8hbbf7', '8hh6n2', '8hj0sw', '8hr1vb', '8hs1o6', '8i15o2',
       '8i2d2p', '8iaar9', '8j5uam', '8jh3qy', '8jpt7o', '8jq8si',
       '8jzqdx', '8koysx', '8kwaqb', '8l5ajs', '8lev7l', '8lnziz',
       '8lxerd', '8m63dg', '8me5s1', '8mlo7f', '8mukqm'], dtype=object)

In [11]:
df['team1'] = df['affiliation']
df['team2'] = df['team1']

The thread_id '8jpt7o' is not for a playoff game but was for the NBA Draft Lottery; we drop this column since it is not relevant to the current analysis.

In [12]:
df = df[df['thread_id'] != '8jpt7o']

In [13]:
df.loc[df['thread_id'].isin(['8ca8gj', '8d0jib', '8drs6h', '8e665k', '8exigg', '8ffe7x']),
       ('team1', 'team2')] = ('Raptors', 'Wizards')

df.loc[df['thread_id'].isin(['8cb74z', '8crv25', '8dijtk', '8dx7i6', '8eosom']), ('team1', 'team2')] = ('Heat', '76ers')

df.loc[df['thread_id'].isin(['8cc2gq', '8d1xau', '8djcz2', '8dy6uc']), ('team1', 'team2')] =  ('Pelicans', 'Blazers')

df.loc[df.thread_id.isin(['8cg1l3', '8d0vrr', '8dsbrb', '8e44h7', '8eod1l', '8f71x0', '8fnfsr']),
       ('team1', 'team2')] = ('Bucks', 'Celtics')

df.loc[df.thread_id.isin(['8ch3y4', '8d9kql', '8drdyv', '8e74eg', '8exiel', '8ffrp7', '8fs7no']), 
       ('team1', 'team2')] = ('Pacers', 'Cavaliers')

df.loc[df.thread_id.isin(['8cifug', '8da05s', '8e00qg', '8egqf2', '8eysfu', '8fgnxe']), 
       ('team1', 'team2')] =  ('Jazz', 'Thunder')

df.loc[df.thread_id.isin(['8cje37', '8daltu', '8dz4uf', '8eft8n', '8eyk43']),
       ('team1', 'team2')] = ('Timberwolves', 'Rockets')

df.loc[df.thread_id.isin(['8csumg', '8djk1s', '8e55jb', '8epudl']), ('team1', 'team2')] = ('Spurs', 'Warriors')    

df.loc[df.thread_id.isin(['8foaac', '8gdo7c', '8h3dc3', '8hh6n2', '8i2d2p']), ('team1', 'team2')] = ('Pelicans', 'Warriors')

df.loc[df.thread_id.isin(['8ft863', '8glq4c', '8h4a1y', '8hj0sw', '8i15o2']), ('team1', 'team2')] = ('Jazz', 'Rockets')

df.loc[df.thread_id.isin(['8g3r8e', '8gut4j', '8ha1um', '8hr1vb', '8iaar9']), ('team1', 'team2')] = ('76ers', 'Celtics')

df.loc[df.thread_id.isin(['8gcq8c', '8gtqly', '8hbbf7', '8hs1o6']), ('team1', 'team2')] = ('Raptors', 'Cavaliers')

df.loc[df.thread_id.isin(['8j5uam', '8jq8si', '8koysx', '8l5ajs', '8lnziz', '8m63dg', '8mlo7f']),
       ('team1', 'team2')] = ('Celtics', 'Cavaliers')

df.loc[df.thread_id.isin(['8jh3qy', '8jzqdx', '8kwaqb', '8lev7l', '8lxerd', '8me5s1', '8mukqm']),
       ('team1', 'team2')] = ('Rockets', 'Warriors')

In [14]:
df['team1'].value_counts()

Jazz            6117
Raptors         5589
Pelicans        4920
Pacers          3964
Rockets         3947
Celtics         3919
Bucks           3288
76ers           2818
Heat            2748
Timberwolves    2744
Spurs           2145
Name: team1, dtype: int64

In [15]:
df['team2'].value_counts()

Cavaliers    10152
Warriors      8871
Celtics       6106
Rockets       5460
Thunder       3401
Wizards       3320
76ers         2748
Blazers       2141
Name: team2, dtype: int64

###  Defining participants and non-participants

I will define `participation` as 1 if the comment's poster was affiliated with one of the teams participating in the game the thread was for and 0 if the comment's poster was affiliated with another team, had a neutral affiliation, or had no reported affiliation.

In [16]:
df['participation'] = df.ups

In [17]:
df.affiliation = df.affiliation.replace(r'\d\b', '', regex=True)

In [18]:
df.participation = np.where(((df['affiliation'] == df['team1']) | (df['affiliation'] == df['team2'])), 1, 0)

In [19]:
df.participation.value_counts()

0    27716
1    14483
Name: participation, dtype: int64

We see that our sample is biased towards the non-participants (about a 2:1 ratio of non-participants to participants). Let's double the number of participant comments by randomly sampling the true data to balance the classes and adding it to the original data:

####  Baseline accuracy

In [20]:
df.participation.mean()

0.34320718500438396

34% of comments were made by fans of participating teams.

## Processing text data

Using regex and pandas' replace method, I remove all punctuation and carriage returns and convert all text to lowercase:

In [21]:
df['text'] = df['text'].replace(r'[^a-zA-Z0-9\s\/]', '', regex=True)

df['text'] = df['text'].map(lambda x: x.lower())

df['text'] = df['text'].replace(r'[\r\n]+', ' ', regex=True)

###  Exporting to csv for modeling

In [22]:
df.to_csv('../data/nbacomments_processed.csv', index=False)